In [ ]:
import json
import uproot
import uproot_methods
import awkward
import numpy as np
from fnal_column_analysis_tools.analysis_objects import JaggedCandidateArray
import fnal_column_analysis_tools.hist as hist
from fnal_column_analysis_tools.hist import plot
%matplotlib nbagg
import matplotlib.pyplot as plt

def nanoObject(tree, prefix):
    branches = set(k.decode('ascii') for k in tree.keys() if k.decode('ascii')[:len(prefix)] == prefix)
    p4branches = [prefix + k for k in ['pt', 'eta', 'phi', 'mass']]
    branches -= set(p4branches)
    objp4 = uproot_methods.TLorentzVectorArray.from_ptetaphim(*[tree[b].array() for b in p4branches])
    branches = {k[len(prefix):]: tree[k].array() for k in branches}
    obj = awkward.JaggedArray.zip(p4=objp4, **branches)
    return obj

In [ ]:
bits = {
    0: 'isPrompt',
    1: 'isDecayedLeptonHadron',
    2: 'isTauDecayProduct',
    3: 'isPromptTauDecayProduct',
    4: 'isDirectTauDecayProduct',
    5: 'isDirectPromptTauDecayProduct',
    6: 'isDirectHadronDecayProduct',
    7: 'isHardProcess',
    8: 'fromHardProcess',
    9: 'isHardProcessTauDecayProduct',
    10: 'isDirectHardProcessTauDecayProduct',
    11: 'fromHardProcessBeforeFSR',
    12: 'isFirstCopy',
    13: 'isLastCopy',
    14: 'isLastCopyBeforeFSR'
}
def statusmask(array, require):
    mask = sum((1<<k) for k,v in bits.items() if v in require)
    return (array & mask)==mask

def getbits(num):
    return [v for k, v in bits.items() if num & (1<<k)]

In [ ]:
with open('files.json') as fin:
    files = json.load(fin)

print("\n".join(files.keys()))

In [ ]:
fvbf = uproot.open(files['VBFHToBB_M-125_13TeV_powheg_pythia8_weightfix'][1])
tvbf = fvbf['Events']
#fggf = uproot.open(files['GluGluHToBB_M-125_13TeV_powheg_MINLO_NNLOPS_pythia8'][29])
fggf = uproot.open("/uscms_data/d3/ncsmith/dazsle/mg5masseffects/nanogen/nanogen.root")
tggf = fggf['Events']

In [ ]:
vbf_genj = nanoObject(tvbf, "GenJet_")
ggf_genj = nanoObject(tggf, "GenJet_")
vbf_genj8 = nanoObject(tvbf, "GenJetAK8_")
ggf_genj8 = nanoObject(tggf, "GenJetAK8_")
vbf_genp = nanoObject(tvbf, "GenPart_")
ggf_genp = nanoObject(tggf, "GenPart_")

In [ ]:
hidx = (ggf_genp['pdgId']==25) & statusmask(ggf_genp['statusFlags'], {'isHardProcess'})
hgen = ggf_genp[hidx][:,0]

In [ ]:
genj = ggf_genj
genj8 = ggf_genj8

hjcross = genj['p4'].cross(genj8[:,:1]['p4'], nested=True)
jcut = (hjcross.i0.delta_r(hjcross.i1) > 0.8).any()
ecut = jcut.sum() >= 2
jcut = jcut & ecut
vbfjets = genj[jcut][:,:2]
detajj = np.abs(vbfjets[:,0:1]['p4'].eta - vbfjets[:,1:2]['p4'].eta)
mjj = (vbfjets[:,0:1]['p4'] + vbfjets[:,1:2]['p4']).mass

In [ ]:
mjj

In [ ]:
cut = (statusmask(ggf_genp['statusFlags'], {'isHardProcess'}) & (ggf_genj.counts==0))

In [ ]:
genj8 = awkward.MaskedArray(ggf_genj8.counts==0, ggf_genj8)

genj8[:,0]

In [ ]:
dr = ggf_genj8[:,:2]['p4'].delta_r(hgen['p4'])
print(np.sum(dr[:,0:1].flatten()<0.8))
print(np.sum(dr[:,1:2].flatten()<0.8))
print(np.sum(dr[:,2:3].flatten()<0.8))
dr = dr[dr.counts==2]
plt.hist2d(dr[:,0].flatten(), dr[:,1].flatten(), alpha=.2)
print(np.sum(dr.regular()<0.8, axis=0))

In [ ]:
hidx = (vbf_genp['pdgId']==25) & statusmask(vbf_genp['statusFlags'], {'isHardProcess'})
hgen = vbf_genp[hidx][:,0]

In [ ]:
hjet = vbf_genj8[:,0]
jcut = vbf_genj['p4'].delta_r(hjet['p4']) > 0.8
jcut = jcut & (jcut.sum()>=2)
vbfjets = vbf_genj[jcut][:,:2]
detajj = np.abs(vbfjets[:,0:1]['p4'].eta - vbfjets[:,1:2]['p4'].eta)
mjj = (vbfjets[:,0:1]['p4'] + vbfjets[:,1:2]['p4']).mass

In [ ]:
plt.hist(detajj.flatten())

In [ ]:
genp = ggf_genp
#mom = genp[genp['genPartIdxMother']]
idx = genp['genPartIdxMother']==0

for ev in genp[idx][:8].tolist():
    print("\nEvent")
    for gp in ev:
        print(gp)
        print("status:", getbits(gp['statusFlags']))

In [ ]:
hidx = (genp['pdgId']==25) & (genp['statusFlags']&(1<<7)).astype(bool)
higgs = genp[hidx]

In [ ]:
heta = hist.Hist("Events", hist.Bin("eta", r"$\eta_h$", 50, -5, 5))
cut = higgs['p4'].pt > 450.
heta.fill(eta=higgs[cut]['p4'].eta.flatten())
plot.plot1d(heta)

In [ ]:
genq = genp[(genp['genPartIdxMother']==0) & (~hidx) & cut.flatten()]
np.unique(genq.counts)

In [ ]:
mjj = (genq['p4'][:,0:1] + genq['p4'][:,1:2]).mass
detajj = np.abs(genq['p4'][:,0:1].eta - genq['p4'][:,1:2].eta)

In [ ]:
vbfcut = ((mjj > 975) & (detajj > 3.25)).sum() == 1
etacut = (np.abs(genq['p4'][:,:2].eta)>2.5).sum() > 0
(etacut & vbfcut).sum() / vbfcut.sum()

In [ ]:
jpt = genq['p4'].pt.pad(3).fillna(0)
jeta = genq['p4'].eta.pad(3).fillna(-100)
hpt = hist.Hist("Events",
                hist.Cat("ijet", "Jet index"),
                hist.Bin("pt", r"$p_{T,j}$", 40, 0, 1000),
                hist.Bin("eta", r"$\eta_j$", 50, -5, 5)
               )
hpt.fill(ijet="0", pt=jpt[:,0].flatten(), eta=jeta[:,0].flatten())
hpt.fill(ijet="1", pt=jpt[:,1].flatten(), eta=jeta[:,1].flatten())
hpt.fill(ijet="2", pt=jpt[:,2].flatten(), eta=jeta[:,2].flatten())
fig, ax, _ = plot.plot2d(hpt.project('ijet', '0'), xaxis='eta')
#ax.set_ylim(1, None)
#ax.set_yscale('log')

In [ ]:
getbits(385)